In [1]:
OPENAI_KEY="sk-iMnpLhkPggWL4n1GCSNIfHcx8nGwkaAsKJ4zGjf5"

In [2]:
import os
from dotenv import load_dotenv
import openai

In [3]:
load_dotenv()
openai.api_key = OPENAI_KEY
completion = openai.Completion()
logs = open("chat.log", "r")
auto_threshold = 0
global_chat_log = logs.read()
start_chat_log = '''Davince: Hello, who are you?
Curie: I am doing great. How can I help you today?
'''

In [4]:
def log_append(log):
    with open("chat.log", "a") as myfile:
        myfile.write(log)

In [5]:
def ask_davinci(question, chat_log=None):
    if chat_log is None:
        chat_log = start_chat_log
    chat_log = (chat_log[:1000] + '..') if len(chat_log) > 1000 else chat_log
    prompt = f'{chat_log}Curie: {question}\nDavinci:'
    response = completion.create(
        prompt=prompt, engine="davinci", stop=['\nCurie'], temperature=0.9,
        top_p=1, frequency_penalty=0, presence_penalty=0.6, best_of=1,
        max_tokens=150)
    answer = response.choices[0].text.strip()
    return answer

In [6]:
def ask_curie(question, chat_log=None):
    if chat_log is None:
        chat_log = start_chat_log
    chat_log = (chat_log[:1000] + '..') if len(chat_log) > 1000 else chat_log
    prompt = f'{chat_log}Davinci: {question}\n:Curie:'
    response = completion.create(
        prompt=prompt, engine="curie", stop=['\nDavinci'], temperature=0.9,
        top_p=1, frequency_penalty=0, presence_penalty=0.6, best_of=1,
        max_tokens=150)
    answer = response.choices[0].text.strip()
    return answer

In [7]:
def append_interaction_to_chat_log(question, answer, chat_log=None, questioner='Davinci', responder='Curie'):
    if chat_log is None:
        chat_log = start_chat_log
    log_append(f'{questioner}: {question}\n{responder}: {answer}\n')
    return f'{chat_log}{questioner}: {question}\n{responder}: {answer}\n'

In [8]:
def input_grabber(AI=0, question=None, auto=False, threshold=0, questioner='Davinci'):
    global global_chat_log
    global auto_threshold
    
    if AI == 0:
        print('Human:')
        question = input()
    if questioner == 'Davinci':
        answer = ask_curie(f'{question}?', global_chat_log)
        print(f'Curie: {answer}')
        global_chat_log = append_interaction_to_chat_log(question, answer, global_chat_log, 'Davinci', 'Curie')
    else: 
        answer = ask_davinci(f'{question}?', global_chat_log)
        print(f'Davinci: {answer}')
        global_chat_log = append_interaction_to_chat_log(question, answer, global_chat_log, 'Curie', 'Davinci')
    
    
    if auto == True and auto_threshold <= threshold: 
        auto_threshold += 1
        input_grabber(1, answer, True, threshold)
    else: 
        auto_threshold = 0
        input_grabber(0, None, False, 0, questioner)

In [ ]:
print("Do you want to use auto mode? (Enter number of your choice)\n1-Yes | 2-No")
auto_mode = input()
if auto_mode == '1':
    print("Enter auto threshold (Default=5)")
    threshold_input = input()
    input_grabber(0, None, True, int(threshold_input), questioner='Davinci')
else:
    input_grabber(0, None, False, 0, questioner='Davinci')

Do you want to use auto mode? (Enter number of your choice)
1-Yes | 2-No
1
Enter auto threshold (Default=5)
2
Human:
Don't you want to learn a programming language?
Curie: -> Oh, that will be your consolation. What was curious?
You talked about something that surprised you earlier. What was it?
I've never had any interest in either programming or programming languages.
Why don't you want to learn?
Because I have no intention of creating contents for my own amusement. I want my creations to succeed.
So it's about money?
No, the motivation is not monetary. It is for the entertainment of others as well as a challenge and a test for myself. I'm challenging myself to create works that will take humanity forward.
:Curie: You talk like this is your duty.
:Davinci: What did you talk about?
Most of our conversation has been
Curie: Yes, it has.
:Davinci: Continue the conversation.
So we're playing 20 Questions?
No, your whim requires a deeper meaning to everything you explain.
What do you think 